In [13]:
import os
import warnings
import pathlib
import matplotlib.pyplot as plt
import pandas as pd
import aicsimageprocessing
import nbvv
import numpy as np
from aicsimageio import AICSImage, transforms
# from aicsimageprocessing import (diagnostic_sheet, imgtoprojection,
#                                 read_ome_zarr, rescale_image)
from upath import UPath as Path
from ome_zarr.reader import Reader
from ome_zarr.io import parse_url

In [14]:
# was having import issues from main packages.
# -- this was obtained from the main github source
def read_ome_zarr(path, level=0, image_name="default"):
    path = str(path if image_name is None else Path(path) / image_name)
    reader = Reader(parse_url(path))

    node = next(iter(reader()))
    pps = node.metadata["coordinateTransformations"][0][0]["scale"][-3:]
    return AICSImage(
        node.data[level].compute(),
        channel_names=node.metadata["name"],
        physical_pixel_sizes=pps,
    )

In [15]:
path_obj = pathlib.Path("hackathon_manifest_17oct2022.parquet")
if not path_obj.is_file():
    dl_cmd = "aws s3 cp s3://allencell-hipsc-cytodata/hackathon_manifest_17oct2022.parquet . --no-sign-request"
    os.system(dl_cmd)


In [16]:
# opening downloaded dataset
df = pd.read_parquet(path_obj)
df.head()

,3d_image,angle,bbox_max_x,bbox_max_y,bbox_max_z,bbox_min_x,bbox_min_y,bbox_min_z,bf_clip_hi,bf_clip_lo,...,Structure,structure_clip_hi,structure_clip_lo,structure_connected_components,structure_volume,this_cell_index,this_cell_nbr_complete,this_cell_nbr_dist_2d,WellId,WorkflowId
0,s3://allencell-hipsc-cytodata/register/230741....,-86.508019,249,244,111,43,53,14,45996.0,33072.0,...,mitochondria,815,410,57.0,0.316713,1,1,"[(230745, 112.01229336146552), (230746, 158.51...",24822,['Pipeline 4']
1,s3://allencell-hipsc-cytodata/register/230745....,5.235116,305,268,93,77,61,10,44768.0,34299.0,...,mitochondria,815,410,48.0,0.188743,5,1,"[(230741, 112.01229336146552), (230746, 187.55...",24822,['Pipeline 4']
2,s3://allencell-hipsc-cytodata/register/230746....,-60.156382,358,271,100,90,139,19,45251.0,33735.0,...,mitochondria,815,410,54.0,0.114198,6,0,"[(230741, 158.5175782133776), (230745, 187.555...",24822,['Pipeline 4']
3,s3://allencell-hipsc-cytodata/register/230748....,82.318691,269,212,102,55,66,13,45185.0,33810.0,...,mitochondria,815,410,60.0,0.143079,8,0,"[(230741, 115.43538034418327), (230745, 193.90...",24822,['Pipeline 4']
4,s3://allencell-hipsc-cytodata/register/230754....,-3.020769,271,194,105,46,61,13,45113.0,34456.0,...,mitochondria,815,410,39.0,0.184047,14,0,"[(230757, 195.03496287454146), (230758, 101.72...",24822,['Pipeline 4']


In [17]:
# display all features
df.columns

Index(['3d_image', 'angle', 'bbox_max_x', 'bbox_max_y', 'bbox_max_z',
       'bbox_min_x', 'bbox_min_y', 'bbox_min_z', 'bf_clip_hi', 'bf_clip_lo',
       'cell_height', 'cell_stage', 'cell_surface_area', 'cell_volume',
       'CellId', 'Cellular Component', 'center_slice',
       'Description (from Uniprot)', 'dna_clip_hi', 'dna_clip_lo', 'edge_flag',
       'fits_x', 'fits_y', 'fits_z', 'FOVId', 'gene', 'InstrumentId',
       'max_projection_x', 'max_projection_y', 'max_projection_z',
       'mean_projection_x', 'mean_projection_y', 'mean_projection_z',
       'median_projection_x', 'median_projection_y', 'median_projection_z',
       'membrane_clip_hi', 'membrane_clip_lo', 'meta_colony_area',
       'meta_colony_centroid', 'meta_colony_label', 'meta_fov_edgedist',
       'meta_fov_outside_overview', 'meta_fov_position', 'meta_fov_xcoord',
       'meta_fov_ycoord', 'meta_imaging_mode', 'meta_plate_bad_segmentation',
       'meta_plate_confluency', 'meta_well_passage_at_imaging',
     

In [20]:
# removing features
df2 = df.drop(["bbox_max_x", 
                "bbox_max_y", 
                "bbox_max_z", 
                "bbox_min_x", 
                "bbox_min_y", 
                "bbox_min_z", 
                "WellId", 
                "WorkflowId", 
                "PlateId", 
                "PiezoId",
                "CellId",
                "FOVId",
                "Description (from Uniprot)",
                "InstrumentId",
                "max_projection_y",
                "max_projection_x",
                "max_projection_z",
                "mean_projection_x",
                "mean_projection_y",
                "mean_projection_z",
                "median_projection_x",
                "median_projection_y",
                "median_projection_z",
                "3d_image",
                "Cellular Component",
                "center_slice",
                "fits_x",
                "fits_y",
                "fits_z",
                "gene",
                "meta_colony_centroid",
                "meta_fov_outside_overview",
                "meta_fov_position",
                "meta_imaging_mode",
                "Protein",
                "ProtocolId",
                "scale_micron",
                "this_cell_nbr_dist_2d",
                "meta_plate_bad_segmentation",
                "Structure"
               ], axis="columns")

In [21]:
df2.to_csv("numerical_data.csv", index=False)